In [1]:
%config Completer.use_jedi = False

In [2]:
import os
import shutil
import geopandas as gpd
import contextily as cx
import pandas as pd
import imageio
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta

In [3]:
stations = pd.read_csv("stations.csv")
stations = gpd.GeoDataFrame(stations, geometry=gpd.points_from_xy(stations.lng, stations.lat), crs='EPSG:4326')
stations

,uid,lat,lng,name,station_id,bike_racks,geometry
0,2585259,52.296298,20.958358,Dewajtis - UKSW,9402,30,POINT (20.95836 52.29630)
1,2585263,52.290974,20.929556,Metro Młociny,9403,30,POINT (20.92956 52.29097)
2,2585265,52.290173,20.950370,Marymoncka - Dewajtis,9404,30,POINT (20.95037 52.29017)
3,2585267,52.285914,20.940561,Metro Wawrzyszew,9405,20,POINT (20.94056 52.28591)
4,2585269,52.233614,21.014635,Górskiego - Szpitalna,9406,16,POINT (21.01463 52.23361)
...,...,...,...,...,...,...,...
309,4393742,52.239409,21.031010,Metro Centrum Nauki Kopernik,9700,30,POINT (21.03101 52.23941)
310,4585378,52.195747,21.002554,Wołoska - Odyńca,9401,16,POINT (21.00255 52.19575)
311,44581598,52.257631,20.919310,Powstańców Śląskich - Wrocławska,9642,16,POINT (20.91931 52.25763)
312,44583449,52.215663,20.960307,Rondo Zesłańców Syberyjskich,9587,26,POINT (20.96031 52.21566)


In [8]:
bikes = pd.read_csv('bikes_assignments_nr.csv')
bikes

,bike_id,station_id,shot,date,state
0,24093,9705,0,2021-11-15,first
1,24093,9705,739,2021-11-14,last
2,24093,9701,1373,2021-11-11,first
3,24093,9701,1401,2021-11-11,last
4,24093,9707,1402,2021-11-11,first
...,...,...,...,...,...
484723,97997,9443,117,2021-11-21,last
484724,97997,9474,118,2021-11-21,first
484725,97997,9474,743,2021-11-21,last
484726,97997,9443,0,2021-11-19,first


In [9]:
# bikes[(bikes['date'] == '2021-11-11') & (bikes['bike_id'] == 97997) ].sort_values('shot')

In [10]:
# bikes = bikes[(bikes['date'] == '2021-11-11') & (bikes['state'] == 'last')].reset_index()
n_shots = max(bikes['shot'])
bikes['time'] = [datetime.strptime(max(bikes['date']), '%Y-%m-%d') + timedelta(days=shot/n_shots) for shot in bikes['shot']]
bikes

KeyboardInterrupt: 

In [11]:
n_shots

5718

In [12]:
24*60*60

86400

In [5]:
plot_settings = {'markersize': 30}
IMG_PATH = 'imgs'
if os.path.exists(IMG_PATH):
    shutil.rmtree(IMG_PATH)
os.makedirs(IMG_PATH, exist_ok=True)
t, i = min(bikes['time']) + timedelta(hours=3), 0
delta  = timedelta(minutes=10)
colors_period = pd.Series(np.zeros(stations.shape[0]))
while t < min(bikes['time']) + timedelta(hours=24):
    fig, ax = plt.subplots(figsize=(30, 20))
    plt.title(t.strftime('%H:%M'))
    stations_period = bikes[(bikes['time']>=t) & (bikes['time']< t+delta)]['station_id']
    colors_period[stations['station_id'].isin(stations_period)] = 1
    ax = stations.plot(ax=ax, c=colors_period.map(lambda x: (1, 1-x, 1-x)), **plot_settings)
    cx.add_basemap(ax=ax, crs=stations.crs.to_string())
    fig.savefig(f'{IMG_PATH}/img{i:0>5}.png', 
              dpi=100, format='png', 
              bbox_inches='tight')
    plt.close()
#     print(t, i, colors_period[:10])
    i += 1
    t += delta
    colors_period *= 0.75

In [7]:
filenames = [os.path.join(IMG_PATH, x ) for x in sorted(os.listdir(IMG_PATH))]

In [8]:
import moviepy.video.io.ImageSequenceClip as vid
clip = vid.ImageSequenceClip(filenames, fps=1)
clip.write_videofile('my_video.mp4')

Moviepy - Building video my_video.mp4.
Moviepy - Writing video my_video.mp4



Moviepy - Done !
Moviepy - video ready my_video.mp4


In [9]:
with imageio.get_writer('movie.gif', mode='I', duration=1) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)